In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
y = train.target
X = train.drop(['target','id'], axis = 1)

In [ ]:
pipeline = Pipeline([('imp', SimpleImputer()),
                   ('xgb', XGBClassifier(tree_method = 'gpu_hist', objective = 'binary:logistic', max_depth = '4',
                       n_estimators = 1000, min_child_weight = 20))])
pipeline.get_params()

In [ ]:
#params = {'xgb__tree_method' : ['gpu_hist'],
#          'xgb__objective' : ['binary:logistic'],
 #         'xgb__max_depth': [4],
  #        'xgb__n_estimators': [1000],
   #       'xgb__min_child_;[20]}
params = {'imp__verbose': [0]}
grid = GridSearchCV(pipeline, param_grid=params, scoring='roc_auc', cv=3)
grid.fit(X,y)
print(pd.DataFrame(grid.cv_results_))

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
test_id = pd.DataFrame(test.id)
test_X = test.drop(['id'], axis=1)
results = grid.predict_proba(test_X)
test_id['target']=results[:,1]
test_id.to_csv('oct_solution.csv', index=False)